In [3]:
import os
import numpy as np
from PIL import Image
from numpy import array
import cv2
float32 = 'float32'

In [16]:
def load_npy(array):
    return np.load(array,allow_pickle=True)
samples_dir = '/home/alex/poker/samples_dir/'
predictions = [samples_dir+i for i in sorted(os.listdir(samples_dir)) if i.endswith('npy')]
a = reformat_answ(load_npy(predictions[0]))

In [18]:
a[-1::]

[{'label': 'Card', 'box': [(320, 507), (395, 621)], 'value': 'd6'}]

In [5]:
    def reformat_answ(input_array):   
        suits = ['♠','♥','♦️','♣','♦','♥','♠','♣'] 
        def area_calc(array):
            # расчет площади ректа
            x1 = int(array[0][0])
            y1 = int(array[0][1])
            x2 = int(array[2][0])
            y2 = int(array[2][1])
            # результат - произведение сторон.
            s = (x2-x1)*(y2-y1)
            return (s)

        def find_suits(array):
            # получение номеров массивом, первый элемент которых - масть.
            #suits = ['♠','♥','♦️','♣','♦','♥','♠','♣'] 
            suits_number = []
            for i in range(len(array)):
                area = area_calc(array[i][1])
                if (array[i][0]) in suits and area > 2000:
                    suits_number += [i]
            return suits_number


        def side_calc(array):
            # расчет длин сторон ректа
            x1 = int(array[0][0])
            y1 = int(array[0][1])
            x2 = int(array[2][0])
            y2 = int(array[2][1])
            return (x2-x1),(y2-y1)

        def big_rect_border(array):
            # функция для определения площади, на которой располагается
            # дублирующа масть в малом ректе для исключения дублирующего ректа.
            #array  = suit_array[1]
            x2 = int(array[1][0])
            # смещаем влево на величину ширины стороны
            x1 = int(array[0][0]) - (int(array[1][0]) - int(array[0][0]))
            y2 = int(array[2][1])
            y1 = int(array[0][1]) - (int(array[2][1]) - int(array[0][1]))    
            return [[x1,x2],[y1,y2]]

        def find_rang(suit_array,input_array, avoid_numbers):
            # ищем позиции, которые могут относится к рангу
            array = suit_array[1]
            array_check_first = array[0]
            # ранг находится по диагонали вверх от верхнего левого угла
            # за точку отчета принимаем верхнюю левую точку ректа
            x = int(array_check_first[0])
            y = int(array_check_first[1])
            answer = 0
            for i in range(len(input_array)):
                # откидываем массивы, которые относятся к малым ректам
                # с мастями
                if i not in avoid_numbers:
                    # массив с которым сравниваем
                    array =  (input_array[i][1])
                    # массив с мастью, которому ищем ранг
                    array_first = array[0]
                    # условие с дельтой по вхождения по
                    # х и у слева и справа
                    if x-40<int(array_first[0])<x+10\
                        and y-70<array_first[1]<y-10\
                        and area_calc(input_array[i][1])>900:
                        #print (i)
                        #p
                        answer = i
            return answer 


        def find_rang_small(suit_array,input_array, avoid_numbers):
            array = suit_array[1]
            array_check_first = array[0]
            x = int(array_check_first[0])
            y = int(array_check_first[1])
             # ранг находится по диагонали вверх от верхнего левого угла
            # за точку отчета принимаем верхнюю левую точку ректа
            x = int(array_check_first[0])
            y = int(array_check_first[1])
            answer = 0
            for i in range(len(input_array)):
                # откидываем массивы, которые относятся к большим ректам,
                # их мастям и малым ректам, которые дублируют большие ректы
                if i not in avoid_numbers:
                    # массив с которым сравниваем
                    array =  (input_array[i][1])
                    # массив с мастью, которому ищем ранг
                    array_first = array[0]
                    # условие с дельтой по вхождения по
                    # х и у слева и справа
                    if x-10<int(array_first[0])<x+10\
                        and y-70<array_first[1]<y-10\
                        and area_calc(input_array[i][1])>600:
                        #print (i)
                        #p
                        answer = i
            return answer 

        def change_suit_to_letter(suit_symbol):
            suit_dict = {'♣':'c', '♦':'d','♥':'h','♠':'s','♦️':'d'}
            return(suit_dict[suit_symbol]) 
     
        
        # определяем ректы с мастями, группируем по площади для исключения дублей
        small_rects = {}
        big_rects = {}
        # большой рект площадь не менее 2000
        for i in range(len(input_array)):
            if input_array[i][0] in suits: 
                #in suits and area_calc(i[1])>2000:
                array = input_array[i]
                # print (i, array[0], area_calc(array[1]))
                if area_calc(array[1])>2000:
                    big_rects[i]=array[0]
                else:
                    small_rects[i]=array[0]
        #print (small_rects,big_rects)
        # исключаем дубли если маленький рект с идентичной мастью находится в площади большого
        # то дропаем его, есди нет - добавляем в add
        add = {}
        for k,v in small_rects.items():
            #print (k,v)
            for a,b in big_rects.items():
                if v == b:
                    #print (k,v,'and',a,b)
                    dx=(big_rect_border(input_array[a][1]))[0]
                    dy=(big_rect_border(input_array[a][1]))[1]
                    x = int(input_array[k][1][2][0])
                    y = int(input_array[k][1][2][1])
                    #print ('x: ',dx[0],x,dx[1])
                    #print ('y: ',dy[0],y,dy[1])
                    if dx[0]<=x<=dx[1] and dy[0]<=y<=dy[1]:
                        #print ('cross')
                        pass
                    else:
                        add[k]=v
                else:
                    add[k]=v
                
        answer = []
        # создаем список индексов, исключаемых из итерации по всему входному аррею
        # сценарий с большими ректами и дублями
        avoid_small = [i for i in small_rects]+[i for i in big_rects]
        for i in big_rects:
            array = input_array[i]
            rang = find_rang(array, input_array, [i for i in small_rects]+\
                            [i for i in big_rects])
            if len(input_array[rang][0]) == 0:
                pass
            else:
                first_point = int(input_array[rang][1][0][0]),int(input_array[rang][1][0][1])
                second_point = int(input_array[i][1][2][0]), int(input_array[i][1][2][1])
                avoid_small += [rang]
                answer+= [[big_rects[i],input_array[rang][0], [first_point,second_point]]]

        to_remove = []
        # индексы исключаются из итреации
        # сценарий с большими. дублями и уникальными малыми ректами
        for i in add: 
            array = input_array[i]
            rang = find_rang_small(array, input_array,avoid_small)
            if len(input_array[rang][0])==0:
                pass
            else:
                first_point = int(input_array[rang][1][0][0]),int(input_array[rang][1][0][1])
                second_point = int(input_array[i][1][2][0]), int(input_array[i][1][2][1])
                to_remove +=[rang]
                answer+= [[add[i],input_array[rang][0], [first_point,second_point]]]   
        # сценарий при отсутствии больших ректов - ранг сопоставляется только с малыми ректами
        if len(big_rects)==0:
            for i in small_rects: 
                array = input_array[i]
                rang = find_rang_small(array, input_array,avoid_small)
                if len(input_array[rang][0])==0:
                    pass
                else:
                    first_point = int(input_array[rang][1][0][0]),int(input_array[rang][1][0][1])
                    second_point = int(input_array[i][1][2][0]), int(input_array[i][1][2][1])
                    to_remove += [rang]
                    answer+= [[small_rects[i],input_array[rang][0], [first_point,second_point]]]  
        # окончательный список индексов - масти и ранги, которые необходимо удалить из входного аррея
        drop_list = to_remove+avoid_small
        all_index = [i for i in range(len(input_array))]
        
        for i in drop_list:
            all_index.remove(i)
            
        old_part = []
        for i in all_index:
            old_part += [input_array[i]]
        # старая часть - арреи, не относящиеся к масти или рангу
        # новая часть - только карты
        new_part = []
        for i in answer:    
            res = {}
            res['label']='Card'
            res['box']=i[2]
            res['value'] = change_suit_to_letter(i[0])+i[1]
            new_part += [res]
            
        return old_part + new_part